#### ¿Que probabilidad de riesgo hay en la regulación de un crédito?
Toda entidad financiera se enfrenta al desafío crítico de gestionar la exposición al riesgo de su cartera de préstamos. El principal problema de negocio es la incertidumbre inherente a la capacidad de repago de los nuevos clientes, lo que impacta directamente en la rentabilidad y la salud financiera de la compañía. Aquí se creará una herramienta que evalúa cada nueva solicitud de préstamo para pronosticar si es probable que el solicitante caiga en impago

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input, State
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier


df_original = pd.read_csv("data/credit_risk.csv")
df_original.info()

# eliminando valores nulos de 'person_emp_length' y 'loan_int_rate'
df_original.dropna(subset=["person_emp_length","loan_int_rate"], inplace=True)

In [ ]:
# creando figura múltiple descriptiva
fig, ax = plt.subplots(2,4, figsize=(20,10))

labels = ["vigente/saldado","impago"]

# primeros 4 gráficos que muestran la distribución en la variables categóricas
sns.countplot(data=df_original, x="person_home_ownership", hue="loan_status", dodge=True, ax=ax[0,0])
ax[0,0].set_title("Tenencia de vivienda del prestatario")
ax[0,0].set_ylabel("Conteo histórico")
ax[0,0].set_xlabel("")
ax[0,0].set_xticks([1,2,3,4])
ax[0,0].set_xticklabels(["alquiler","dueño","hipoteca","otro"])
legend_0 = ax[0,0].legend(title="estado del préstamo", labels=labels, fontsize=8)

sns.countplot(data=df_original, x="loan_intent", hue="loan_status", dodge=True, ax=ax[0,1])
ax[0,1].set_title("Propósito del préstamo")
ax[0,1].set_ylabel("")
ax[0,1].set_xlabel("")
ax[0,1].set_xticks([1,2,3,4,5,6])
ax[0,1].set_xticklabels(["personal","educación","médico","emresarial","hogar(mejoras)","deudas(consolidación)"])
ax[0,1].tick_params(axis="x", labelsize=8, labelrotation=45)
legend_1 = ax[0,1].legend(title="estado del préstamo", labels=labels, fontsize=8)

sns.countplot(data=df_original, x="loan_grade", hue="loan_status", dodge=True, ax=ax[0,2])
ax[0,2].set_title("Grado del préstamo")
ax[0,2].set_ylabel("")
ax[0,2].set_xlabel("")
legend_2 = ax[0,2].legend(title="estado del préstamo", labels=labels, fontsize=8)

sns.countplot(data=df_original, x="cb_person_default_on_file", hue="loan_status", dodge=True, ax=ax[0,3])
ax[0,3].set_title("Incumplimientos de pago")
ax[0,3].set_ylabel("")
ax[0,3].set_xlabel("")
ax[0,3].set_xticks([1,2])
ax[0,3].set_xticklabels(["si","no"])
legend_3 = ax[0,3].legend(title="estado del préstamo", labels=labels, fontsize=8)

plt.setp(legend_0.get_title(), fontsize=8)
plt.setp(legend_1.get_title(), fontsize=8)
plt.setp(legend_2.get_title(), fontsize=8)
plt.setp(legend_3.get_title(), fontsize=8)

# últimos 4 gráficos que muestra la dispersión de las variables numéricas (seleccionadas para posibles relaciones)
sns.scatterplot(data=df_original, x="loan_status", y="loan_percent_income", ax=ax[1,0])
ax[1,0].set_title("Mayor porcentage posible mayor riesgo")
ax[1,0].set_ylabel("porcentage del préstamo sobre el ingreso")
ax[1,0].set_xlabel("0 = vigente/saldado       1 = impago")

sns.scatterplot(data=df_original, x="person_income", y="loan_amnt", hue="loan_status", ax=ax[1,1])
ax[1,1].set_title("Mayor ingreso posible mayor monto")
ax[1,1].set_ylabel("monto total")
ax[1,1].set_xlabel("ingreso anual")
ax[1,1].ticklabel_format(axis="x", style="plain")
ax[1,1].tick_params(axis="x", labelsize=8, labelrotation=45)
legend_4 = ax[1,1].legend(title="estado del préstamo", labels=labels, fontsize=8)

sns.scatterplot(data=df_original, x="person_age", y="person_emp_length", hue="loan_status", alpha=0.7, ax=ax[1,2])
ax[1,2].set_title("Menor edad posible mayor riesgo")
ax[1,2].set_ylabel("años empleado")
ax[1,2].set_xlabel("edad")
legend_5 = ax[1,2].legend(title="estado del préstamo", labels=labels, fontsize=8)

sns.scatterplot(data=df_original, x="cb_person_cred_hist_length", y="loan_int_rate", hue="loan_status", ax=ax[1,3])
ax[1,3].set_title("Mayor historial posible menor interés")
ax[1,3].set_ylabel("tasa de interés")
ax[1,3].set_xlabel("historial de crédito (años)")
legend_6 = ax[1,3].legend(title="estado del préstamo", labels=labels, fontsize=8)

plt.setp(legend_4.get_title(), fontsize=8)
plt.setp(legend_5.get_title(), fontsize=8)
plt.setp(legend_6.get_title(), fontsize=8)

plt.subplots_adjust(hspace=0.5, wspace=0.3)
plt.show()

#### Feature Engireering
es el proceso de crear nuevas variables (o features) a partir de los datos existentes, utilizando el conocimiento del negocio y la lógica, con el objetivo de mejorar la capacidad predictiva de un modelo de machine learning.

- One Hot Encoding en variables categóricas: es una técnica que convierte las variables categóricas en un formato numérico que creando nuevas columnas binarias (solo 0s y 1s) para cada categoría única, de manera que el algoritmo no asuma erróneamente un orden o jerarquía, lo cual ocurriría si simplemente asignaras números ordinales y trata cada categoría como una entidad independiente, asegurando una representación precisa e imparcial de las variables.

- PCA en variables numéricas: es una técnica estadística que se utiliza para la reducción de dimensionalidad. Su función es transformar un conjunto grande de variables numéricas relacionadas en un conjunto más pequeño de nuevas variables, llamadas componentes principales, que capturan la mayor parte de la información (varianza) de los datos originales, eliminando la multicolinealidad y el ruido. 

Este enfoque de preprocesamiento de datos se basa en un enfoque dual diseñado para optimizar el rendimiento y la estabilidad del modelo. Al combinar ambas técnicas logramos un balance óptimo, ya que trabajan en conjuntos de datos separados (categóricos vs. numéricos) para preparar un conjunto final altamente informativo, limpio y computcacionalmente eficiente.

In [ ]:
# eliminando valores absurdos de las variables 'person_age' y 'person_emp_length'
df_original = df_original.loc[(df_original["person_age"] < 100) | (df_original["person_emp_length"] < 100),:]

# generando una copia del dataset original
df = df_original.copy()
df.reset_index(drop=True, inplace=True)

# separando las variables predictoras (X) de la variable objetivo (Y)
X = df.drop(["loan_status","loan_intent","loan_grade","person_home_ownership","cb_person_default_on_file"], axis=1)
y = df["loan_status"] 

# estandarizando los datos (esto centra los datos a media=0 y desviación estándar=1)
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

# inicializando PCA sin un número específico de componentes para ver cuánta varianza explica cada uno
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# analizando cuánta "información" captura cada nuevo componente principal
explained_variance = pca.explained_variance_ratio_
print("Varianza explicada por cada componente:")
print(explained_variance)
print(f"Suma acumulada de varianza con los primeros 3 componentes: {np.sum(explained_variance[:3]):.2f}")

# visualizando la Varianza Acumulada para decidir cuántos componentes usar
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(explained_variance) + 1), np.cumsum(explained_variance), marker="o", linestyle="--")
plt.title("Varianza Explicada Acumulada por Componente Principal")
plt.xlabel("Número de Componentes")
plt.ylabel("Varianza Acumulada Explicada")
plt.grid(True)
plt.show()

In [ ]:
pca = PCA(n_components=2)

# reduciendo los datos estandarizados a 2 dimensiones
pca.fit(X_scaled)
X_pca = pca.transform(X_scaled)

columns_pca = ["PC1_numeric","PC2_numeric"]
df_pca = pd.DataFrame(X_pca, columns=columns_pca)

categorical_cols = ["loan_intent","loan_grade","person_home_ownership","cb_person_default_on_file"]

one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
one_hot_encoder.fit(df[categorical_cols])
cols_encoded = one_hot_encoder.transform(df[categorical_cols])

new_cols_names = one_hot_encoder.get_feature_names_out(categorical_cols)
df_one_hot = pd.DataFrame(cols_encoded, columns=new_cols_names)

# uniendo ambos data frames junto con la variable objetivo
df_processed = pd.concat([df_one_hot, df_pca, df["loan_status"]], axis=1)

df_processed

#### Bagging (Bootstrap Aggregating)
es una de las principales técnicas computacionales de estadística inferencial cuya objetivo se basa en eliminar o reducir lo máximo posible la varianza en los resultados, su método es utilizar la sabiduría de las masas, es decir, reforzar el aprendizaje mediante el criterio de distintos metaestimadores que se encargarán de sacar sus propias conclusiones de respectivos datos seleccionados para luego, dependiendo del problema en cuestión, llegar a una respuesta final a través de la media o moda del conjunto de resultados. Al entrenar múltiples modelos con datos ligeramente distintos y contar sus resultados, el Bagging suaviza estas fluctuaciones. La agregación de los resultados cancela la varianza individual de cada modelo, resultando en un modelo de conjunto con un error de varianza significativamente menor.

In [ ]:
# algoritmo de clasificación KNN
knn_classifier = KNeighborsClassifier(n_neighbors=5)

# metaestimador de clasificación
bagging_knn = BaggingClassifier(estimator=knn_classifier, # clasificador base
                                n_estimators=100, # cantidad de estimadores
                                max_samples=0.3,  # número de muestras requeridas para cada estimador
                                bootstrap=True) # muestreo con reemplazo

bagging_knn.fit(df_processed[df_processed.columns[:-1]], df["loan_status"])

# nuevo objeto aleatorio extraído del conjunto
object = df.sample(n=1)

# codificando las variables categóricas del objeto
object_categorical_cols = object[categorical_cols]
categorical_cols_encoded = one_hot_encoder.transform(object_categorical_cols)
categorical_cols_encoded = pd.DataFrame(categorical_cols_encoded, columns=new_cols_names)

# reduciendo la dimensión de las varibales numéricas del nuevo objeto
categorical_cols.append("loan_status")
object_numeric_cols = object.drop(categorical_cols, axis=1)
numeric_cols_scaled = scaler.transform(object_numeric_cols)
numeric_cols_pca = pca.transform(numeric_cols_scaled)
numeric_cols_pca = pd.DataFrame(numeric_cols_pca, columns=columns_pca)

# nuevo objeto procesado
object_processed = pd.concat([categorical_cols_encoded, numeric_cols_pca], axis=1)

# prediciendo la clase (loan_status) de dicho objeto
predict_encoded = bagging_knn.predict(object_processed)[0]

# extrayendo la probabilidad del modelo para la clase de ese objeto
predict_proba = bagging_knn.predict_proba(object_processed)
probability = predict_proba[0, predict_encoded]*100

if predict_encoded == 0:
    probability = 100 - probability 

probability = str(probability)

# mostrando el nuevo objeto, su clase y la probabilidad de que pertenezca
print("-------------------------------------")
print("nuevo préstamo solicitado")
print(f"\npredicción: riesgo de impago de {probability[:4]}%")
print("-------------------------------------")

#### Dashboard que ejerce como detector de riesgos de créditos para nuevos préstamos solicitados

In [ ]:
df_processed = df_processed.loc[(df_processed["PC1_numeric"] < 15),:]

default = df_processed.loc[df_processed["loan_status"] == 1,:]
non_default = df_processed.loc[df_processed["loan_status"] == 0,:]

probability_text = html.B(id="probability",children=[],style={})
colors = ("green","red")

graph_pca = go.Figure()
graph_pca.add_trace(go.Scatter(x=default["PC1_numeric"], y=default["PC2_numeric"], mode="markers", marker_color="red", name="en default"))
graph_pca.add_trace(go.Scatter(x=non_default["PC1_numeric"], y=non_default["PC2_numeric"], mode="markers", marker_color="green", name="regularizado"))
graph_pca.update_layout(title="Crédito regularizado vs. en default")
graph_pca.update_layout(legend=dict(font=dict(size=9)))

app = dash.Dash(__name__)

app.layout =  html.Div(id="body",className="e4_body",children=[
    html.H1("Evaluación en riesgo de crédito",id="title",className="e4_title"),
    html.Div(id="dashboard",className="e4_dashboard",children=[
        html.Div(className="e4_graph_div",children=[
            dcc.Graph(id="graph_pca",className="e4_graph",figure=graph_pca),
            html.Form(id="input_div",className="input_div",children=[
                dcc.Input(id="input_1",className="input",type="text",placeholder="Edad",size="7"),
                dcc.Input(id="input_2",className="input",type="text",placeholder="Ingreso anual",size="7"),
                dcc.Input(id="input_3",className="input",type="text",placeholder="Tenencia de vivienda",size="7"),
                dcc.Input(id="input_4",className="input",type="text",placeholder="Longitud empleado",size="7"),
                dcc.Input(id="input_5",className="input",type="text",placeholder="Intención del préstamo",size="7"),
                dcc.Input(id="input_6",className="input",type="text",placeholder="Grado del préstamo",size="7"),
                dcc.Input(id="input_7",className="input",type="text",placeholder="Monto total",size="7"),
                dcc.Input(id="input_8",className="input",type="text",placeholder="Interés del préstamo",size="7"),
                dcc.Input(id="input_9",className="input",type="text",placeholder="Porcentage sobre el ingreso",size="7"),
                dcc.Input(id="input_10",className="input",type="text",placeholder="Historial de incumplimientos",size="7"),
                dcc.Input(id="input_11",className="input",type="text",placeholder="Historial de crédito(años)",size="7"),
                html.Button("enviar",id="button",type="button",className="input_button",n_clicks=0)
            ]),
            html.P(["predicción: riesgo de impago de ",probability_text,"%"],className="e4_predict")
        ])
    ])
])
        
@app.callback(
    [Output(component_id="graph_pca",component_property="figure"),
    Output(component_id="probability",component_property="children"),
    Output(component_id="probability",component_property="style")],
    [Input(component_id="button",component_property="n_clicks")],
    [State(component_id="input_1",component_property="value"),
    State(component_id="input_2",component_property="value"),
    State(component_id="input_3",component_property="value"),
    State(component_id="input_4",component_property="value"),
    State(component_id="input_5",component_property="value"),
    State(component_id="input_6",component_property="value"),
    State(component_id="input_7",component_property="value"),
    State(component_id="input_8",component_property="value"),
    State(component_id="input_9",component_property="value"),
    State(component_id="input_10",component_property="value"),
    State(component_id="input_11",component_property="value")]
)

def update_graph(n_clicks, var_1, var_2, var_3, var_4, var_5, var_6, var_7, var_8, var_9, var_10, var_11):
    if n_clicks is not None and n_clicks > 0:
        
        object = pd.DataFrame({
            "person_age":[var_1],
            "person_income":[var_2],
            "person_home_ownership":[var_3],
            "person_emp_length":[var_4],
            "loan_intent":[var_5],
            "loan_grade":[var_6],
            "loan_amnt":[var_7],
            "loan_int_rate":[var_8],
            "loan_percent_income":[var_9],
            "cb_person_default_on_file":[var_10],
            "cb_person_cred_hist_length":[var_11]
        })
           
        categorical_cols = ["loan_intent","loan_grade","person_home_ownership","cb_person_default_on_file"]   
        object_categorical_cols = object[categorical_cols]
        categorical_cols_encoded = one_hot_encoder.transform(object_categorical_cols)
        categorical_cols_encoded = pd.DataFrame(categorical_cols_encoded, columns=new_cols_names)

        object_numeric_cols = object.drop(categorical_cols, axis=1)
        numeric_cols_scaled = scaler.transform(object_numeric_cols)
        numeric_cols_pca = pca.transform(numeric_cols_scaled)
        numeric_cols_pca = pd.DataFrame(numeric_cols_pca, columns=columns_pca)

        object_processed = pd.concat([categorical_cols_encoded, numeric_cols_pca], axis=1)

        predict_encoded = bagging_knn.predict(object_processed)[0]
        predict_proba = bagging_knn.predict_proba(object_processed)
        probability = predict_proba[0, predict_encoded]*100
        
        if predict_encoded == 0:
            probability = 100 - probability
            if probability >= 45:
                predict_encoded = 1
        
        probability = str(probability)
        probability = probability[:4]
        probability_color = {"color":colors[predict_encoded]}
        
        graph_pca.add_trace(go.Scatter(x=object_processed["PC1_numeric"], y=object_processed["PC2_numeric"], mode="markers", marker_color="blueviolet", name="nuevo préstamo"))
    
    return graph_pca, probability, probability_color


if __name__ == "__main__":
    app.run_server(debug=False)